<a href="https://colab.research.google.com/github/gsenas/deeplearning/blob/master/Flowers_Datagen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import os

_URL = 'https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz'

zip_file = tf.keras.utils.get_file(origin=_URL, fname='flower_photos.tgz', extract=True, cache_subdir='/content')

base_dir = os.path.join(os.path.dirname(zip_file),'flower_photos')

IMAGE_SIZE = 224
BATCH_SIZE = 64

datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

train_generator = datagen.flow_from_directory(base_dir,target_size=(IMAGE_SIZE, IMAGE_SIZE), batch_size=BATCH_SIZE, subset='training')

val_generator = datagen.flow_from_directory(base_dir,target_size=(IMAGE_SIZE, IMAGE_SIZE), batch_size=BATCH_SIZE, subset='validation')

print(train_generator.class_indices)

labels  = '\n'.join(sorted(train_generator.class_indices.keys()))

with open('labels.txt','w') as f:
  f.write(labels)

Found 2939 images belonging to 5 classes.
Found 731 images belonging to 5 classes.
{'daisy': 0, 'dandelion': 1, 'roses': 2, 'sunflowers': 3, 'tulips': 4}


In [ ]:
IMG_SHAPE = (IMAGE_SIZE, IMAGE_SIZE, 3)

base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE, include_top=False, weights='imagenet')

9412608/9406464 [==============================] - 1s 0us/step


In [ ]:
base_model.trainable = False

In [ ]:
model = tf.keras.Sequential([
  base_model,
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.GlobalAveragePooling2D(),
  tf.keras.layers.Dense(5, activation='softmax')
])

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(),loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
epochs = 10

history = model.fit(train_generator, epochs=epochs,validation_data=val_generator)

Epoch 1/10
46/46 [==============================] - 24s 342ms/step - loss: 1.3646 - accuracy: 0.6025 - val_loss: 0.4068 - val_accuracy: 0.8413
Epoch 2/10
46/46 [==============================] - 14s 303ms/step - loss: 0.2835 - accuracy: 0.9039 - val_loss: 0.3574 - val_accuracy: 0.8769
Epoch 3/10
46/46 [==============================] - 14s 303ms/step - loss: 0.1908 - accuracy: 0.9326 - val_loss: 0.3895 - val_accuracy: 0.8782
Epoch 4/10
46/46 [==============================] - 14s 302ms/step - loss: 0.1201 - accuracy: 0.9589 - val_loss: 0.3770 - val_accuracy: 0.8700
Epoch 5/10
46/46 [==============================] - 14s 303ms/step - loss: 0.0762 - accuracy: 0.9796 - val_loss: 0.4488 - val_accuracy: 0.8646
Epoch 6/10
46/46 [==============================] - 14s 302ms/step - loss: 0.0486 - accuracy: 0.9894 - val_loss: 0.5278 - val_accuracy: 0.8440
Epoch 7/10
46/46 [==============================] - 14s 299ms/step - loss: 0.0404 - accuracy: 0.9899 - val_loss: 0.4751 - val_accuracy: 0.8632

In [ ]:
saved_model_dir=''
tf.saved_model.save(model, saved_model_dir)

converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
tflite_model = converter.convert()

with open('model.tflite','wb') as f:
  f.write(tflite_model)

INFO:tensorflow:Assets written to: assets


In [ ]:
from google.colab import files

files.download('model.tflite')
files.download('labels.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>